In [ ]:
import pandas as pd
from itertools import combinations
from collections import Counter

# Carregando a base de dados com nomes reais dos produtos na primeira linha
base = pd.read_csv('PaoManteiga.csv', sep=';', encoding='cp1252', header=0)
nomes_produtos = list(base.columns)

# 1️⃣ Gerar transações com base apenas nas colunas com 'Sim'
transacoes = []
for i in range(len(base)):
    transacao = []
    for j in range(base.shape[1]):
        if base.values[i, j] == 'Sim':
            transacao.append(nomes_produtos[j])
    transacoes.append(transacao)

# Total de transações
n_transacoes = len(transacoes)

# 2️⃣ ITEMSET 1 - Suporte de cada item individual
itens = Counter()
for transacao in transacoes:
    for item in transacao:
        itens[item] += 1

itemset1 = {}
print("ITEMSET 1 (Suportes):")
for item, count in itens.items():
    suporte = count / n_transacoes
    itemset1[item] = suporte
    print(f"{item}: suporte = {suporte:.2f}")
print()

# 3️⃣ ITEMSET 2 - Regras 2 a 2 (considerando direções opostas)
itemset2 = {}
print("ITEMSET 2 (Regras 2 a 2 com suporte >= 0.3 e confiança >= 0.8):")
for a, b in combinations(itemset1.keys(), 2):
    # A => B
    count_ab = 0
    count_a = 0
    for transacao in transacoes:
        if a in transacao:
            count_a += 1
            if b in transacao:
                count_ab += 1
    suporte_ab = count_ab / n_transacoes
    confianca_ab = count_ab / count_a if count_a else 0
    if suporte_ab >= 0.3 and confianca_ab >= 0.8:
        itemset2[(a, b)] = (suporte_ab, confianca_ab)
        print(f"{a} => {b}: suporte = {suporte_ab:.2f}, confiança = {confianca_ab:.2f}")

    # B => A
    count_ba = 0
    count_b = 0
    for transacao in transacoes:
        if b in transacao:
            count_b += 1
            if a in transacao:
                count_ba += 1
    suporte_ba = count_ba / n_transacoes
    confianca_ba = count_ba / count_b if count_b else 0
    if suporte_ba >= 0.3 and confianca_ba >= 0.8:
        itemset2[(b, a)] = (suporte_ba, confianca_ba)
        print(f"{b} => {a}: suporte = {suporte_ba:.2f}, confiança = {confianca_ba:.2f}")
print()

# 4️⃣ ITEMSET 3 - Regras 3 a 3 considerando todas as combinações de 3 itens
print("ITEMSET 3 (Regras 3 a 3 com suporte >= 0.3 e confiança >= 0.8):")
itemset3 = {}
itens_validos = set()
for par in itemset2.keys():
    itens_validos.update(par)

for trio in combinations(itens_validos, 3):
    A, B, C = trio
    count_abc = 0
    for transacao in transacoes:
        if A in transacao and B in transacao and C in transacao:
            count_abc += 1
    suporte_abc = count_abc / n_transacoes

    if suporte_abc >= 0.3:
        # A, B => C
        count_ab = sum(1 for t in transacoes if A in t and B in t)
        if count_ab:
            confianca = count_abc / count_ab
            if confianca >= 0.8:
                print(f"{A}, {B} => {C}: suporte = {suporte_abc:.2f}, confiança = {confianca:.2f}")
        # A, C => B
        count_ac = sum(1 for t in transacoes if A in t and C in t)
        if count_ac:
            confianca = count_abc / count_ac
            if confianca >= 0.8:
                print(f"{A}, {C} => {B}: suporte = {suporte_abc:.2f}, confiança = {confianca:.2f}")
        # B, C => A
        count_bc = sum(1 for t in transacoes if B in t and C in t)
        if count_bc:
            confianca = count_abc / count_bc
            if confianca >= 0.8:
                print(f"{B}, {C} => {A}: suporte = {suporte_abc:.2f}, confiança = {confianca:.2f}")
        # A => B, C
        count_a = sum(1 for t in transacoes if A in t)
        if count_a:
            confianca = count_abc / count_a
            if confianca >= 0.8:
                print(f"{A} => {B}, {C}: suporte = {suporte_abc:.2f}, confiança = {confianca:.2f}")
        # B => A, C
        count_b = sum(1 for t in transacoes if B in t)
        if count_b:
            confianca = count_abc / count_b
            if confianca >= 0.8:
                print(f"{B} => {A}, {C}: suporte = {suporte_abc:.2f}, confiança = {confianca:.2f}")
        # C => A, B
        count_c = sum(1 for t in transacoes if C in t)
        if count_c:
            confianca = count_abc / count_c
            if confianca >= 0.8:
                print(f"{C} => {A}, {B}: suporte = {suporte_abc:.2f}, confiança = {confianca:.2f}")


ITEMSET 1 (Suportes):
Café: suporte = 0.30
Pão: suporte = 0.50
Manteiga: suporte = 0.50
Leite: suporte = 0.20
Cerveja: suporte = 0.20
Feijão: suporte = 0.20
Arroz: suporte = 0.20

ITEMSET 2 (Regras 2 a 2 com suporte >= 0.3 e confiança >= 0.8):
Café => Pão: suporte = 0.30, confiança = 1.00
Café => Manteiga: suporte = 0.30, confiança = 1.00
Pão => Manteiga: suporte = 0.40, confiança = 0.80
Manteiga => Pão: suporte = 0.40, confiança = 0.80

ITEMSET 3 (Regras 3 a 3 com suporte >= 0.3 e confiança >= 0.8):
Manteiga, Café => Pão: suporte = 0.30, confiança = 1.00
Café, Pão => Manteiga: suporte = 0.30, confiança = 1.00
Café => Manteiga, Pão: suporte = 0.30, confiança = 1.00
